In [1]:
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, GRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from models_def import Attention

# prepare other feat
fl = [
    '../features/other_feat.pkl',
    '../features/lgb1_feat.pkl',
    '../features/lr_feat1.pkl',
    '../features/lr_feat2.pkl',
    '../features/ridge_feat1.pkl',
    '../features/ridge_feat2.pkl',
    '../features/mnb_feat1.pkl',
    '../features/mnb_feat2.pkl',
    '../features/wordbatch_feat.pkl',
    '../features/tilli_lr_feat.pkl',
    '../features/fasttext_cnn2d_5_feat.pkl',
'../features/fasttext_cnn_gru_5_feat.pkl',
'../features/fasttext_cnn_v1_5_feat.pkl',
'../features/fasttext_cnn_v2_5_feat.pkl',
'../features/fasttext_cudnn_gru_5_feat.pkl',
'../features/fasttext_gru_v1_5_feat.pkl',
'../features/fasttext_lstm_v1_5_feat.pkl',

]
def get_feat(f):
    with open(f,'rb') as fin:
        a,b = pickle.load(fin)
        return a,b

# load feats
train_x,test_x = [],[]
for feat in fl:
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
print(train_x[0])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
print(train_x[0])

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


file path ../features/other_feat.pkl
(159571, 37) (153164, 37)
file path ../features/lgb1_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/ridge_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/ridge_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/wordbatch_feat.pkl
(159571, 6) (153164, 6)
file path ../features/tilli_lr_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../featu

In [2]:
max_features = 160000
maxlen = 250

# Contraction replacement patterns
cont_patterns = [
    (b'(W|w)on\'t', b'will not'),
    (b'(C|c)an\'t', b'can not'),
    (b'(I|i)\'m', b'i am'),
    (b'(A|a)in\'t', b'is not'),
    (b'(\w+)\'ll', b'\g<1> will'),
    (b'(\w+)n\'t', b'\g<1> not'),
    (b'(\w+)\'ve', b'\g<1> have'),
    (b'(\w+)\'s', b'\g<1> is'),
    (b'(\w+)\'re', b'\g<1> are'),
    (b'(\w+)\'d', b'\g<1> would'),
    (b'&lt;3', b' heart '),
    (b':d', b' smile '),
    (b':dd', b' smile '),
    (b':p', b' smile '),
    (b'8\)', b' smile '),
    (b':-\)', b' smile '),
    (b':\)', b' smile '),
    (b';\)', b' smile '),
    (b'\(-:', b' smile '),
    (b'\(:', b' smile '),
    (b'yay!', b' good '),
    (b'yay', b' good '),
    (b'yaay', b' good '),
    (b':/', b' worry '),
    (b':&gt;', b' angry '),
    (b":'\)", b' sad '),
    (b':-\(', b' sad '),
    (b':\(', b' sad '),
    (b':s', b' sad '),
    (b':-s', b' sad '),
    (b'\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3}', b' '),
    (b'(\[[\s\S]*\])', b' '),
    (b'[\s]*?(www.[\S]*)', b' ')
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]



def new_clean(text):
    """ Simple text clean up process"""
    # 1. Go to lower case (only good for english)
    # Go to bytes_strings as I had issues removing all \n in r""
    clean = bytes(text.lower(), encoding="utf-8")
    
    # replace words like hhhhhhhhhhhhhhi with hi
    for ch in string.ascii_lowercase:
        pattern = bytes(ch+'{3,}', encoding="utf-8")
        clean = re.sub(pattern, bytes(ch, encoding="utf-8"), clean)
    # 2. Drop \n and  \t
    clean = clean.replace(b"\n", b" ")
    clean = clean.replace(b"\t", b" ")
    clean = clean.replace(b"\b", b" ")
    clean = clean.replace(b"\r", b" ")
    # 3. Replace english contractions
    for (pattern, repl) in patterns:
        clean = re.sub(pattern, repl, clean)
    # 4. Drop puntuation
    # I could have used regex package with regex.sub(b"\p{P}", " ")
    exclude = re.compile(b'[%s]' % re.escape(bytes(string.punctuation, encoding='utf-8')))
    clean = b" ".join([exclude.sub(b'', token) for token in clean.split()])
    # 5. Drop numbers - as a scientist I don't think numbers are toxic ;-)
    clean = re.sub(b"\d+", b" ", clean)
    # 6. Remove extra spaces - At the end of previous operations we multiplied space accurences
    clean = re.sub(b'\s+', b' ', clean)
    # Remove ending space if any
    clean = re.sub(b'\s+$', b'', clean)
    # 7. Now replace words by words surrounded by # signs
    # e.g. my name is bond would become #my# #name# #is# #bond#
    # clean = re.sub(b"([a-z]+)", b"#\g<1>#", clean)
    clean = re.sub(b" ", b"# #", clean)  # Replace space
    clean = b"#" + clean + b"#"  # add leading and trailing #

    return str(clean, 'utf-8')

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return new_clean(text)

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 6)
(159571, 250) (153164, 250)
rebelion 71283
proverb 21036
ecn 70849
diamter 135075
jerkwads 107383
libeling 32578
2000000
Preparing embedding matrix
Null word embeddings: 66521


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU,CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import gc


def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model(comp):
    inp = Input(shape=(maxlen, ))
    inp_2 = Input(shape=[train_x.shape[1]], name="other")
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = SpatialDropout1D(0.4)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    att = Attention(maxlen)(x)
    conc = concatenate([att,avg_pool, max_pool,inp_2])
    # conc = Dropout(0.1)(conc)
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=[inp,inp_2], outputs=outp)
    if comp:
        model.compile(loss='binary_crossentropy',
                      optimizer='nadam',
                      metrics=['accuracy'])

    return model
print('def model done')

def model done


In [4]:
m = get_model(False)
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 300)     48000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 250, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 250, 256)     330240      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
attention_

In [ ]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        curr_other_x = train_x[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        hold_out_other_x = train_x[test_index]
        
        # model
        model = get_model(True)
        batch_size = 64
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit([curr_x,curr_other_x], curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=([hold_out_x,hold_out_other_x],hold_out_y), 
                  callbacks=callbacks_list)
        
        model.load_weights(file_path)
        y_test = model.predict([X_test,test_x])
        test_pred += y_test
        hold_out_pred = model.predict([hold_out_x,hold_out_other_x])
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [ ]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

train_pred,test_pred = kf_train(fold_cnt=5,rnd=4)
print(train_pred.shape,test_pred.shape)    

sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/pool_gru1_fasttext_adj2_sample_5.gz", index=False, compression='gzip')
with open('../features/pool_gru_fasttext_adj2_5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')


# fold4 output feat
# final 0.0401879770356 0.984252568867, pre model, not good

# test add other feat, NON-NN feat to nn
# add first 3 files, 3891 1st fold
# add more non-nn feats, 3883 1st fold
# final 0.03855334827205233 0.9847883805119143

# len 250, max word 160000, more text clean
# 1st fold, 3876
# final 0.038284512738446075 0.9849492284521206

# add more other feat, lr1, mnb1, ridge, 1st fold 3831, 3824, 3729, 3804, 3839
# final 0.03805630898087786 0.9848301591976821

# add no pretrained, bottom dr 0.1
# 1st fold 3908

# rm bottom dr, 1st fold 3796 , later worse

# add fasttext nn other model, 3821, 3846, 3726

Train on 127656 samples, validate on 31915 samples
Epoch 1/8
127656/127656 [==============================] - 155s 1ms/step - loss: 0.0454 - acc: 0.9835 - val_loss: 0.0398 - val_acc: 0.9845
Epoch 2/8
127656/127656 [==============================] - 155s 1ms/step - loss: 0.0388 - acc: 0.9851 - val_loss: 0.0386 - val_acc: 0.9847
Epoch 3/8
127656/127656 [==============================] - 159s 1ms/step - loss: 0.0374 - acc: 0.9855 - val_loss: 0.0382 - val_acc: 0.9849
Epoch 4/8
127656/127656 [==============================] - 160s 1ms/step - loss: 0.0363 - acc: 0.9857 - val_loss: 0.0396 - val_acc: 0.9844
Epoch 5/8
127656/127656 [==============================] - 161s 1ms/step - loss: 0.0352 - acc: 0.9860 - val_loss: 0.0412 - val_acc: 0.9835
Epoch 6/8
127656/127656 [==============================] - 161s 1ms/step - loss: 0.0341 - acc: 0.9864 - val_loss: 0.0394 - val_acc: 0.9845
Epoch 7/8
127656/127656 [==============================] - 161s 1ms/step - loss: 0.0331 - acc: 0.9869 - val_loss: 0

In [ ]:
train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)
print(train_pred.shape,test_pred.shape)    

sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/pool_gru1_fasttext_adj2_sample_10.gz", index=False, compression='gzip')
with open('../features/pool_gru_fasttext_adj2_10_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')
# final 0.037889400552850935 0.9849074497663528 PUB 9865

# add more other feat, lr1, mnb1, ridge
# final 0.03770786578985529 0.9850045852107631